In [ ]:
#모듈 import
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas  as pd 
import psycopg2
from datetime import date, timedelta

driver = webdriver.Chrome('C:\chromedriver.exe')

#크롬드라이버 실행
driver.get('http://www.naver.com')
time.sleep(2)

#키워드를 검색한 네이버창 접속
query_txt = '뷔스티에 원피스'
element = driver.find_element_by_id("query")
element.send_keys(query_txt)
element.submit()

# 뷰 클릭
driver.find_element_by_link_text("VIEW").click()
time.sleep(2)

#필터 클릭
driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
time.sleep(2)

#기간 1년 설정
driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()

no2 = [ ]           # 게시글 번호 컬럼
title2 = [ ]        # 게시물 제목 컬럼
bdate2 = [ ]        # 작성 일자 컬럼
url2 = [ ]          # 블로그 url 주소

no = 1

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)


#더이상 스크롤이 안되면 종료
i = 1
time.sleep(1)

#현재 스크롤 위치를 last_height로 받아옴
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    #스크롤 한 번 실행 후 스크롤 위치를 new_height로 받아옴
    scroll_down(driver)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")

    #더이상 스크롤이 안되면 종료
    if new_height == last_height:
        break

    last_height = new_height
    i += 1


print("\n")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

view_list = soup.find('ul','lst_total').find_all('li')

for i in view_list :
    
    no2.append(no)                            # 게시물 번호 리스트에 추가
    print('진행 중:',no)

    all_title = i.find_all('a')
    title = all_title[5].get_text( )          # 게시물 제목
    title2.append(title)                      # 게시물 제목 리스트에 추가

    bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
    bdate.replace('.', '-')
    bdate2.append(bdate)                     # 작성일자 리스트에 추가

    url = str(all_title[0])
    url = url.split('"')[5]
    url2.append(url)

    if no == 200 : #한번에 200개 까지만 크롤링 가능
        break
        
    no += 1

print('포스트 개수 :', no)

bdate3 = [] #전처리 후 작성일자

# 작성일자를 날짜형식으로 통일화
for day in bdate2 :

    if '시간 전' in day :       # n시간 전 업로드 글
        day = date.today()
    elif '일 전' in day :       # n일 전 업로드 글 
        day = date.today() - timedelta(int(day[0])) #8일전 글 부터는 날짜로 반환
    elif day == '어제' :        # 어제 업로드 글
        day = date.today() - timedelta(1)
    else :
        day = day.rstrip('.')
        day = day.replace('.', '-')
    
    bdate3.append(day)

# df화
naver_blog = pd.DataFrame()
naver_blog['제목'] = title2
naver_blog['작성일자'] = bdate3
naver_blog['주소'] = url2

#date-time 타입으로 변환
naver_blog['작성일자'] = pd.to_datetime(naver_blog['작성일자'])

print(naver_blog.head())

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url) values (%s, %s, %s);'
    return command

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()


table = input('테이블명 입력 :')

#table에 데이터 넣기
for i in range(len(naver_blog)) :
    cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2]))
    conn.commit()



In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [ ]:
#db 연결 종료
conn.cursor().close()
conn.close()